In [1]:
#Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn import svm
from scipy.stats import norm
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from typing import Union
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from IPython.display import display
from time import time
import matplotlib.widgets
from matplotlib.widgets import RadioButtons, CheckButtons
%matplotlib nbagg 
import matplotlib.animation 
from sklearn.inspection import permutation_importance
import random
import pickle

# Pre-processing 

In order to clean the data provided, each of the different features were researched and considered. Ultimately, we have decided to remove the 'In literature', 'Compound', 'v(A)', 'v(B)' and 'tau'. The 'In literature' feature was removed because it didn't give useful information for the crystal structure. Then, the 'Compound' feature was removed because 'A' and 'B' would already define the elements present in the compound. Lastly, 'v(A)' and 'v(B)' which were the oxidation states were removed due to a considerable amount of missing values, and as a result the ‘tau’ column has too been removed. This is because the ‘tau’ value is one calculated based off of the oxidation state values and therefore has the same issues. An oxidation state of zero is also unreasonable because this only occurs when an element is a neutral substance containing only atoms of one element.

Next, the 'A' and 'B' features were One Hot encoded to transform the categorical data. In order to avoid a hierarchical ordering, the One Hot Encoder was chosen. For this encoding method, each class is replaced by a boolean column, instead of simply being ranked by an integer.

Ultimately, after the appropriate data had been selected, a scaler was applied. It was determined that the min-max scaler was the most appropriate. This is because the standard scaler has a variance of 1.0, and therefore the limits cannot be known beforehand. This is not the case with the min-max scaler, and is why that it has been chosen to be used for our purposes.

In [2]:
# Data cleaning 
data = pd.read_csv('Crystal_structure.csv')
data_to_clean = data.copy()
data_without_columns = data_to_clean.drop(columns= ['In literature','v(A)','v(B)','τ', 'Compound'])
data_without_rows = data_without_columns.dropna()
data_replace_dash = data_without_rows.replace('-',np.nan)
data_replace_zero = data_replace_dash.replace(0, np.nan)
data_replace = data_replace_zero.dropna()

# Encoding 
class Encoder:
    def __init__(self, kind: str = 'onehot'):
        # make sure kind is either onehot or label
        assert kind in ['onehot', 'label']
        self.kind = kind
        
    def encode(self, data: pd.Series) -> Union[pd.DataFrame, pd.Series]:
        if self.kind == 'onehot':
            categories = set(data)
            new = pd.DataFrame()
            for column in list(categories):
                new[column] = data.apply(lambda x: 1 if x == column else 0)
        else:
            categories = list(set(data))
            new = data.apply(lambda x: categories.index(x))
            new = pd.DataFrame(new)
            del new[column]
        return new


ohe_encoded = data_replace.copy()
a = Encoder('onehot').encode(ohe_encoded['A'])
b = Encoder('onehot').encode(ohe_encoded['B'])
del ohe_encoded['A']
del ohe_encoded['B']
ohe_encoded = pd.concat([ohe_encoded, a], axis=1)
ohe_encoded = pd.concat([ohe_encoded, b], axis=1)


# Min-max scaling 
def min_max_scaling(data, min_idx=0, max_idx=10):
    data_minmax = data.copy()
    name_columns = list(data_minmax.columns)
    for i in range(min_idx, max_idx):  
        min_value = min(data_minmax[name_columns[i]])
        max_value = max(data_minmax[name_columns[i]])
        diff = int(max_value) - int(min_value)
        data_minmax[name_columns[i]] = data_minmax[name_columns[i]].apply(lambda x: (x - min_value) / diff) 
    return data_minmax

data_minmax = min_max_scaling(ohe_encoded, 0, 10)

# Training, testing and classifiers

The data was divided into training and testing sets with the most common split being 70/30, respectively. A higher portion for the training set was used in large data set, therefore, we chose a 75/25 split. In addition, random_state shuffles the data around and is commonly set to 42, which is we decided to use. 

Four differing classifications were tested to then find the best classifier, namely: multinomial logistic regression, random forest, k nearest neighbours and support vectors machine. They were compared using the score function, which determined how well the data was fitted.

For logistic regression and random forest, the LogisticRegression() and RandomForestClassifier() classes of the scikit-learn library were used. For K nearest neighbours regression, the scores of the different values of K were determined and the K value with the highest score was used. The range was put up to slightly higher than the number of data points as this is commonly seen as the ideal K value. Lastly, for the support vector machine, both the linear and the kernel versions were used to find the optimal classifier. A C value of 10 was chosen to obtain a smoother decision surface. Gamma was chosen to be 1, which is an intermediate value. A lower value would cause a lower accuracy and higher would be at risk of overfitting.

In [3]:
# Test-train split 
name_columns = list(data_minmax.columns)
X = data_minmax[name_columns[:10]].to_numpy()
y = data_minmax[name_columns[10]].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

# Logistic regression 
logistic = LogisticRegression(max_iter = 1000).fit(X_train, y_train)
logistic_y_pred = logistic.predict(X_test)
logistic_score = logistic.score(X_test, y_test)
print('Logistic Regression Score: %.2f' % (logistic_score*100))

#Random forest 
forest = RandomForestClassifier().fit(X_train,y_train)
forest_y_pred = forest.predict(X_test)
forest_score = forest.score(X_test, y_test)
print('Random Forest Score: %.2f' % (forest_score*100))

#KNN
models = []
k_values = list(range(1,70))
# Testing different k values 
for k in k_values:
    model= KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    models.append(model)
scores = [model.score(X_test, y_test) for model in models]
# Find k value with highest score 
highest_score = max(scores)
index = scores.index(highest_score)
# Train KNN model
knn_model = KNeighborsClassifier(n_neighbors=k_values[index]).fit(X_train, y_train)
knn_y_pred = knn_model.predict(X_test)
knn_score = knn_model.score(X_test, y_test)
print('KNN Score: %.2f' % (knn_score*100))

#SVM 
C = 10.0
# Testing different SVM models 
models = (
    svm.SVC(kernel='linear', C=C),
    svm.LinearSVC(C=C, max_iter=10000),
    svm.SVC(kernel='rbf', gamma=1, C=C),
    svm.SVC(kernel='poly', degree=1.5, gamma='auto', C=C)
)
models = [clf.fit(X_train, y_train) for clf in models]
scores = [clf.score(X_test, y_test) for clf in models]
# Get SVM model with highest score 
highest_score = max(scores)
index = scores.index(highest_score)
# Train SVM model 
svm_y_pred = models[index].predict(X_test)
svm_score = models[index].score(X_test, y_test)
print('SVM Score: %.2f' % (svm_score*100))   

Logistic Regression Score: 63.35
Random Forest Score: 78.80
KNN Score: 75.92
SVM Score: 74.35


In [4]:
# Export models and data to be used in next notebook
forest_filename = 'forest_model.sav'
pickle.dump(forest, open(forest_filename, 'wb'))
pd.DataFrame(X_test).to_csv("X_test.csv")
pd.DataFrame(y_test).to_csv("y_test.csv")
pd.DataFrame(X_train).to_csv("X_train.csv")
pd.DataFrame(y_train).to_csv("y_train.csv")
pd.DataFrame(forest_y_pred).to_csv("forest_y_pred.csv")
pd.DataFrame(knn_y_pred).to_csv("knn_y_pred.csv")
pd.DataFrame(X).to_csv("X.csv")
pd.DataFrame(y).to_csv("y.csv")
pd.DataFrame(name_columns).to_csv("name_columns.csv")
data_minmax.to_csv("data_minmax.csv")